In [1]:
import pandas as pd
import scipy.io
import numpy as np
from scipy.io import wavfile

In [6]:
def write_voice(filename, data, fs):
    
    df = pd.DataFrame(data)
    wavfile.write(filename=filename, rate=fs, data=np.array(df).astype(np.int16))
    return None

In [3]:
mat = scipy.io.loadmat('VDE_i_head_3B.mat')
channel_i = mat['vde3_i']

mat = scipy.io.loadmat('VDE_q_head_3B.mat')
channel_q = mat['vde3_q']

In [33]:
sample_rate = 6000
fs = 134400
actual_sample_rate = int(fs / sample_rate)

channel_ii = channel_i[0]
channel_qq = channel_q[0]

print(len(channel_ii))

#channel_ii = channel_ii[::sample_rate]
#channel_qq = channel_qq[::sample_rate]
data = {'1': channel_ii, '2': channel_qq}

write_voice('VDE_3B.wav',{'1': channel_ii,'2': channel_qq}, sample_rate)

6762624


In [10]:
import numpy as np
import pandas as pd
import scipy

import matplotlib.pyplot as plt
from scipy.io import wavfile

%matplotlib inline

In [11]:
def load_music(filename):
    '''
    Takes the location of the music WAV file as a input and returns 
    the sampling frequency (Hz) of the sound file, the time array 
    of the digital sound signal (sec), and the sound channels.
    
    inputs:
        filename (str)
    outputs:
        fs (int)
        t List[float]
        channels List[List[float]]
    '''
    fs, data = wavfile.read(filename)
    num_channels = np.shape(data)[1]
    
    L = np.shape(data)[0]
    t = np.array(range(0, L))/fs

    channels = {}
    for n in range(num_channels):
        channels[str(n+1)] = [data[i][n] for i in range(L)]
    
    return fs, t, channels

def write_music(filename, data, fs):
    '''
    Writes the sound data into a WAV file with location filename. 
    The sampling frequency is given in Hz, and the data is given as
    a dictionary of which the values are the channels in the music file.
    
    inputs:
        filename (str)
        data Dict{any: List[float]}
        fs (int)
    outputs:
        None
    '''
    df = pd.DataFrame(data)
    wavfile.write(filename=filename, rate=fs, data=np.array(df).astype(np.int16))
    return None

def freq_spectrum(data, fs, lim=None):
    '''
    Plots the frequency spectrum of music channels with sampling frequency fs (Hz).
    
    inputs:
        data List[float]
        fs (int)
        lim (optional to limit plots x-axis) (float)
    outputs:
        None
    '''
    N = np.size(data)

    yf = fft(data) # fft of the signal 

    xf = fftfreq(N, 1/fs)
    xf = fftshift(xf) #  preparing the x-axis for the spectrum

    yplot = fftshift(yf)

    plt.plot(xf, 1.0/N * np.abs(yplot))
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude')
    if lim is not None:
        plt.xlim([-lim,lim])
    plt.grid()
    return None

In [34]:
fs, t, channels = load_music('morganFreemanResized.wav')
channel1 = np.array(channels['1'])
channel2 = np.array(channels['2'])

N = np.logical_and(0 < t, t <= 5) # trimming a second from it
t = t[N]
t = t - t[0]
channel1 = channel1[N]
channel2 = channel2[N]

print(len(channel1))

30000


In [27]:
sample_rate = 6000
actual_sample_rate = int(fs/sample_rate)
# Resampled signal
channel1_resampled = channel1[::actual_sample_rate]
channel2_resampled = channel2[::actual_sample_rate]
write_music('morganFreemanResampled.wav',{'1': channel1_resampled,'2':channel2_resampled},sample_rate)